# Network Deliberation ABM
## Discrete state, generated NK Model task

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches': None}

# Imports
import configparser
import math

import matplotlib as mpl
from matplotlib import pyplot as plt
import netdelib.soclearn as slearn
import numpy as np
import pandas as pd
import repsci

# Configure plotting in Jupyter
from matplotlib import pyplot as plt

plt.rcParams['font.size'] = 18
plt.rcParams['axes.linewidth'] = 2

symbols = ['.','.','.','.']
colors = ['#0000ff','#999966','#dddd77', '#aaaaff']
plt.figure(figsize=(7.5,10))
exclude = ['Random', 'StochasticBlock', 'Lattice']
reproduce = [
    '2021-01-06 130248 bc84b67'
]

In [ ]:
def load_result_list(experiments):
    strategies = []
    networks = []
    
    score = {}
    correct = {}
    distance = {}
    initial_distance = {}
    converge_time = {}
    
    for experiment_id in experiments:
        experiment, exp_score, exp_correct, exp_distance, exp_initial, exp_converge = load_results(experiment_id)
        
        exp_strategies = experiment.config.get('strategies', 'enabled').split(',')
        for i, s in enumerate(exp_strategies):
            
            if s not in strategies:
                strategies.append(s)
                score[s] = {}
                correct[s] = {}
                distance[s] = {}
                initial_distance[s] = {}
                converge_time[s] = {}
            
            exp_networks = experiment.config.get('networks', 'enabled').split(',')
            for j, n in enumerate(exp_networks):
                
                if n not in networks:
                    networks.append(n)
                
                score[s][n] = exp_score[i][j]
                correct[s][n] = exp_correct[i][j]
                distance[s][n] = exp_distance[i][j]
                if exp_initial is not None:
                    initial_distance[s][n] = exp_initial[i][j]
                if exp_converge is not None:
                    converge_time[s][n] = exp_converge[i][j]
    
    return experiment, strategies, networks, score, correct, distance, initial_distance, converge_time
    
def load_results (experiment_id):
    
    experiment = repsci.Experiment(
        "discrete_generated_nk",
        reproduce=experiment_id)
    config = experiment.get_config()
    
    score = np.load(experiment.get_filename('score.npy'))
    correct = np.load(experiment.get_filename('correct.npy'))
    distance_score = np.load(experiment.get_filename('distance_score.npy'))
    # Older code did not produce the below data
    try:
        initial_distance = np.load(experiment.get_filename('initial_distance.npy'))
    except FileNotFoundError:
        initial_distance = None
    try:
        converge_time = np.load(experiment.get_filename('converge_time.npy'))
    except FileNotFoundError:
        converge_time = None
    return experiment, score, correct, distance_score, initial_distance, converge_time

def individual_title(config):
    individual_mode = config.getint('strategies', 'individual_mode', fallback=1)
    if config.getboolean('strategies', 'individual', fallback=False):
#        if config.getboolean('strategies', 'individual_all_bits', fallback=True):
#            title_individual = 'Hill-Climb'
#        else:
#            title_individual = 'Mutation'
        title_individual = ''
        if individual_mode == slearn.MODE_ALL:
            title_individual += 'Serial-Indiv.'
        elif individual_mode == slearn.MODE_FALLBACK:
            title_individual += 'Fallback-Indiv.'
        elif individual_mode == slearn.MODE_BEST:
            title_individual += 'Parallel-Indiv.'
    else:
        title_individual = 'social-only'
    return title_individual

def result_title (config):
    individual_mode = config.getint('strategies', 'individual_mode', fallback=1)
    if config.getboolean('strategies', 'individual', fallback=False):
        if config.getboolean('strategies', 'individual_all_bits', fallback=True):
            title_individual = 'hill-climb'
        else:
            title_individual = 'mutation'
        if individual_mode == slearn.MODE_ALL:
            title_individual += ' serial indiv.'
        elif individual_mode == slearn.MODE_FALLBACK:
            title_individual += ' fallback indiv.'
        elif individual_mode == slearn.MODE_BEST:
            title_individual += ' parallel indiv.'
    else:
        title_individual = 'social-only'
    title = "|V|={} N={}, K={}, EXP={}, S={}, {}, {}".format(
        config.getint('abm', 'N'),
        config.getint('abm', 'bit_count'),
        config.getint('abm', 'K'),
        config.getfloat('abm', 'nk_exponent', fallback=1),
        config.getint('strategies', 'sample', fallback=None),
        {True: 'critical', False: 'non-critical'}[config.getboolean('strategies', 'critical')],
        title_individual)
    return title

def nested_label(data, fig, ax, width, height=1, offset=(0, 0), ):

    label_trans = mpl.transforms.blended_transform_factory(
        fig.transFigure, ax.transAxes)

    layer_width = width[0]
    cell_height = height / len(data)
    
    for i, data in enumerate(reversed(data)):
        rect = mpl.patches.Rectangle(
            (offset[0] + lw, offset[1] + i * cell_height), width=layer_width, height=cell_height,
            lw=2, edgecolor='black', facecolor="white",
            transform=label_trans, clip_on=False)
#        plt.text(
#            offset[0] + 0.02, offset[1] + i * cell_height + 0.04, data['label'],
#            fontsize=18,
#            transform=label_trans, rotation='vertical')
        plt.text(
            offset[0] + 0.02, offset[1] + (i + 0.5) * cell_height, data['label'],
            fontsize=18, verticalalignment='center',
            transform=label_trans, rotation='vertical')
        ax.add_patch(rect)
        if 'data' in data:
            inner_offset = (
                offset[0] + layer_width,
                offset[1] + (i) * cell_height)
            nested_label(data['data'], fig, ax, width[1:], height=cell_height, offset=inner_offset)


In [ ]:
experiment, learning_strategies, networks, score, correct, distance_score, initial_distance, converge_time = load_result_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

plt.figure(figsize=(6.66, 7.77))

for i, strat in enumerate(learning_strategies):
    if strat == 'Local Majority':
        continue
    plt.subplot(len(learning_strategies) - 1, 1,1+i)
    xlabels = []
    ys = []
    err95s = []
    for j, net in enumerate(reversed(networks)):
        if net in exclude:
            continue
        xlabels.append(net)
        y = score[strat][net][0]
        t, best = max(enumerate(y), key=lambda x: x[1])
        ys.append(best)
        err95 = 1.96 * np.array(score[strat][net][1]) / math.sqrt(runs)
        err95s.append(err95[t])
    plot_title = strat 
    if plot_title == 'Local Majority':
        plot_title = 'Bitwise Majority'
    plot_title += ' + ' + individual_title(experiment.config)
    plt.title(plot_title, fontsize=18)
    plt.errorbar(
        ys, range(len(xlabels)), xerr=err95s,
        fmt='{}'.format(symbols[i]),
        markerfacecolor='{}ff'.format(colors[0]), color=colors[0], markeredgecolor='{}ff'.format(colors[0]),
        markersize=0,
        capsize=8,
        label=strat)
    plt.xlim([0, 1])
    plt.ylim([-0.5, 3.5])
    plt.grid(True)
    plt.yticks(ticks=range(len(xlabels)), labels=xlabels, rotation=0)
    plt.xticks(fontsize=16)
#    if i == 0:
plt.xlabel('Mean Solution Quality')


plt.tight_layout()
out = experiment.get_filename('performance.png')
plt.savefig(out, dpi=600)
plt.savefig(experiment.get_filename('performance.eps'), dpi=600)


In [ ]:
num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

plt.figure(figsize=(6.66,7.77))

for i, strat in enumerate(learning_strategies):
    if strat == 'Local Majority':
        continue
    plt.subplot(len(learning_strategies) - 1, 1,1+i)
    xlabels = []
    ys = []
    err95s = []
    for j, net in enumerate(reversed(networks)):
        if net in exclude:
            continue
        xlabels.append(net)
        y, std = converge_time[strat][net]
        ys.append(y)
        err95 = 1.96 * np.array(std) / math.sqrt(runs)
        err95s.append(err95)
    plot_title = strat 
    if plot_title == 'Local Majority':
        plot_title = 'Bitwise Majority'
    plot_title += ' + ' + individual_title(experiment.config)
    plt.title(plot_title, fontsize=18)
    plt.errorbar(
        ys, range(len(xlabels)), xerr=err95s,
        fmt='{}'.format(symbols[i]),
        markerfacecolor='{}ff'.format(colors[0]), color=colors[0], markeredgecolor='{}ff'.format(colors[0]),
        markersize=0,
        capsize=8,
        label=strat)
    plt.ylim([-0.5, 3.5])
    plt.xlim([1, 220])
    plt.xscale('log')
    plt.grid(True)
    plt.yticks(ticks=range(len(xlabels)), labels=xlabels, rotation=0)
    plt.xticks(fontsize=16)
#    if i == 0:
#plt.ylabel('Mean Final Score', fontsize=20)

plt.xlabel('Convergence Time')

#plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1), fontsize=12)
plt.tight_layout()
out = experiment.get_filename('convergence.png')
plt.savefig(out, dpi=600)
plt.savefig(experiment.get_filename('convergence.eps'), dpi=600)


In [ ]:
plt.figure(figsize=(10,7.5))
exclude = ['Random', 'StochasticBlock', 'Lattice']

for i, strat in enumerate(learning_strategies):
    plt.subplot(2, 2,1+i)
    xlabels = []
    ys = []
    err95s = []
    for j, net in enumerate(reversed(networks)):
        if net in exclude:
            continue
        y = score[strat][net][0]
        err95 = 1.96 * np.array(initial_distance[strat][net][1]) / math.sqrt(runs)
        plt.plot(range(len(y)), y, label=net, color=colors[j])
    plot_title = strat 
    if plot_title == 'Local Majority':
        plot_title = 'Bitwise Majority'
    plot_title += ' + ' + individual_title(experiment.config)
    plt.title(plot_title, fontsize=16)
    plt.grid(True)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.legend(loc='lower right', fontsize=12)
    plt.xlim([0, 75])
    plt.ylim([0, 0.75])
    plt.ylabel('Mean dist. from any initial belief (bits)')
    plt.xlabel('Iteration')
#    if i == 0:
#plt.ylabel('Mean Final Score', fontsize=20)

plt.tight_layout()
out = experiment.get_filename('initial_distance.png')
plt.savefig(out, dpi=600)
plt.savefig(experiment.get_filename('initial_distance.eps'), dpi=600)


In [ ]:
fig = plt.figure(figsize=(6.66, 12))
ax = plt.gca()

data = [
    {'label': 'Parallel', 'data': []},
    {'label': 'Fallback', 'data': []}
]

xlabels = []

reproduce = [
    '2021-01-06 130248 bc84b67'
]


experiment, learning_strategies, networks, score, correct, distance_score, initial_distance, converge_time = load_result_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

row = 0

for i, strat in enumerate(reversed(learning_strategies)):
    if strat == 'Local Majority':
        continue
    ys = []
    err95s = []
    for j, net in enumerate(reversed(networks)):
        
        # Skip excluded networks
        if net in exclude:
            continue
        
        # Add labels and update to newer terminology
        xlabel = net
        if xlabel == 'Random Group':
            xlabel = 'Random Pod'
        xlabels.append(xlabel)
        
        y = score[strat][net][0]
        t, best = max(enumerate(y), key=lambda x: x[1])
        ys.append(best)
        err95 = 1.96 * np.array(score[strat][net][1]) / math.sqrt(runs)
        err95s.append(err95[t])

    plot_title = strat 
    if plot_title == 'Local Majority':
        plot_title = 'Bitwise Majority'
    plot_title = plot_title.replace(' ', "\n")
    # We're going bottom to top, so insert on beginning of list
    data[0]['data'] = [{'label': plot_title}] + data[0]['data']

    plt.errorbar(
        ys, range(row, row + len(ys)), xerr=err95s,
        fmt='{}'.format(symbols[i]),
        markerfacecolor='{}ff'.format(colors[0]), color=colors[0], markeredgecolor='{}ff'.format(colors[0]),
        markersize=0,
        capsize=8,
        label=strat)
    plt.axhline(row - 0.5, color='#000000')
    row += 1 * len(ys)

    
reproduce = [
    '2021-01-11 173220 bc84b67'
]

experiment, learning_strategies, networks, score, correct, distance_score, initial_distance, converge_time = load_result_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

for i in range(6):
    plt.fill_between([0, 1], i * 4 - 0.5, i * 4 + 1.5, color="#e0e0ff")

for i, strat in enumerate(reversed(learning_strategies)):
    if strat == 'Local Majority':
        continue
    ys = []
    err95s = []
    for j, net in enumerate(reversed(networks)):
        
        # Skip excluded networks
        if net in exclude:
            continue
        
        # Add labels and update to newer terminology
        xlabel = net
        if xlabel == 'Random Group':
            xlabel = 'Random Pod'
        xlabels.append(xlabel)
        
        y = score[strat][net][0]
        t, best = max(enumerate(y), key=lambda x: x[1])
        ys.append(best)
        err95 = 1.96 * np.array(score[strat][net][1]) / math.sqrt(runs)
        err95s.append(err95[t])

    plot_title = strat 
    if plot_title == 'Local Majority':
        plot_title = 'Bitwise Majority'
    plot_title = plot_title.replace(' ', "\n")
    # We're going bottom to top, so insert on beginning of list
    data[1]['data'] = [{'label': plot_title}] + data[1]['data']

    plt.errorbar(
        ys, range(row, row + len(ys)), xerr=err95s,
        fmt='{}'.format(symbols[i]),
        markerfacecolor='{}ff'.format(colors[0]), color=colors[0], markeredgecolor='{}ff'.format(colors[0]),
        markersize=0,
        capsize=8,
        label=strat)
    plt.axhline(row - 0.5, color='#000000')
    row += 1 * len(ys)

plt.xlim([0, 1])
plt.ylim([-0.5, len(xlabels) - 0.5])
plt.yticks(ticks=range(len(xlabels)), labels=xlabels, rotation=0)
plt.xticks(fontsize=16)
plt.xlabel('Solution Quality')

plt.subplots_adjust(left=0.455, right=0.95, top=0.95)
left, bottom = fig.transFigure.inverted().transform(
    ax.transData.transform_point((0,0)))
right, top = fig.transFigure.inverted().transform(
    ax.transData.transform_point((1,1)))    

right_margin = 0.2675
lw = 0.01
width = left - right_margin - lw
nested_label(data, fig, ax, [width / 3, width * 2 / 3])

for i in range(6):
    label_trans = mpl.transforms.blended_transform_factory(
        fig.transFigure, ax.transAxes)

    h = 1 / 6
    
    rect = mpl.patches.Rectangle(
        (left - right_margin, i / 6), width=right_margin, height=h,
        lw=2, edgecolor='black', facecolor="none",
        transform=label_trans, clip_on=False)
    ax.add_patch(rect)

axt = ax.secondary_xaxis('top')
axt.tick_params(labelsize=18)
plt.grid(True, axis='x', which='both')

out = experiment.get_filename('performance.png')
plt.savefig(out, dpi=600)
plt.savefig(experiment.get_filename('performance.eps'), dpi=600)


In [ ]:
fig = plt.figure(figsize=(6.66, 12))
ax = plt.gca()

data = [
    {'label': 'Parallel', 'data': []},
    {'label': 'Fallback', 'data': []}
]

xlabels = []

reproduce = [
    '2021-01-06 130248 bc84b67'
]


experiment, learning_strategies, networks, score, correct, distance_score, initial_distance, converge_time = load_result_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

row = 0

for i, strat in enumerate(reversed(learning_strategies)):
    if strat == 'Local Majority':
        continue

    ys = []
    err95s = []
    for j, net in enumerate(reversed(networks)):

        if net in exclude:
            continue

        # Add labels and update to newer terminology
        xlabel = net
        if xlabel == 'Random Group':
            xlabel = 'Random Pod'
        xlabels.append(xlabel)

        y, std = converge_time[strat][net]
        ys.append(y)
        err95 = 1.96 * np.array(std) / math.sqrt(runs)
        err95s.append(err95)
        
    plot_title = strat 
    if plot_title == 'Local Majority':
        plot_title = 'Bitwise Majority'
    plot_title = plot_title.replace(' ', "\n")
    # We're going bottom to top, so insert on beginning of list
    data[0]['data'] = [{'label': plot_title}] + data[0]['data']

    plt.errorbar(
        ys, range(row, row + len(ys)), xerr=err95s,
        fmt='{}'.format(symbols[i]),
        markerfacecolor='{}ff'.format(colors[0]), color=colors[0], markeredgecolor='{}ff'.format(colors[0]),
        markersize=0,
        capsize=8,
        label=strat)    
    row += 1 * len(ys)
    plt.axhline(row - 0.5, color='#000000')
    
reproduce = [
    '2021-01-11 173220 bc84b67'
]

experiment, learning_strategies, networks, score, correct, distance_score, initial_distance, converge_time = load_result_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

for i, strat in enumerate(reversed(learning_strategies)):
    if strat == 'Local Majority':
        continue

    ys = []
    err95s = []
    for j, net in enumerate(reversed(networks)):

        if net in exclude:
            continue

        # Add labels and update to newer terminology
        xlabel = net
        if xlabel == 'Random Group':
            xlabel = 'Random Pod'
        xlabels.append(xlabel)

        y, std = converge_time[strat][net]
        ys.append(y)
        err95 = 1.96 * np.array(std) / math.sqrt(runs)
        err95s.append(err95)
        
    plot_title = strat 
    if plot_title == 'Local Majority':
        plot_title = 'Bitwise Majority'
    plot_title = plot_title.replace(' ', "\n")
    # We're going bottom to top, so insert on beginning of list
    data[1]['data'] = [{'label': plot_title}] + data[1]['data']

    plt.errorbar(
        ys, range(row, row + len(ys)), xerr=err95s,
        fmt='{}'.format(symbols[i]),
        markerfacecolor='{}ff'.format(colors[0]), color=colors[0], markeredgecolor='{}ff'.format(colors[0]),
        markersize=0,
        capsize=8,
        label=strat)    
    row += 1 * len(ys)
    plt.axhline(row - 0.5, color='#000000')

plt.subplots_adjust(left=0.455, right=0.95, top=0.95)
left, bottom = fig.transFigure.inverted().transform(
    ax.transData.transform_point((0,0)))
right, top = fig.transFigure.inverted().transform(
    ax.transData.transform_point((1,1)))    
left = 0.455

right_margin = 0.2675
lw = 0.01
width = left - right_margin - lw
nested_label(data, fig, ax, [width / 3, width * 2 / 3])

for i in range(6):
    label_trans = mpl.transforms.blended_transform_factory(
        fig.transFigure, ax.transAxes)

    h = 1 / 6
    
    rect = mpl.patches.Rectangle(
        (left - right_margin, i / 6), width=right_margin, height=h,
        lw=2, edgecolor='black', facecolor="none",
        transform=label_trans, clip_on=False)
    ax.add_patch(rect)

for i in range(6):
    plt.fill_between([1, 200], i * 4 - 0.5, i * 4 + 1.5, color="#e0e0ff")

axt = ax.secondary_xaxis('top')
plt.xscale('log')
plt.grid(True, axis='x')
plt.yticks(ticks=range(len(xlabels)), labels=xlabels, rotation=0)
plt.xticks(fontsize=16)
plt.xlabel('Iterations to Converge')
plt.xlim([1, 200])
plt.ylim([-0.5, 23.5])

out = experiment.get_filename('convergence.png')
plt.savefig(out, dpi=600)
plt.savefig(experiment.get_filename('convergence.eps'), dpi=600)
